## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [2]:
# File location and type
file_location = "/FileStore/tables/amazon_alexa.csv"
file_type = "csv"
   
# CSV options
infer_schema = "true" #cambiar a true
first_row_is_header = "true" #cambiar a true
delimiter = ","
# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [3]:
display(df.head(10))

rating,date,variation,verified_reviews,feedback
5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
5,31-Jul-18,Charcoal Fabric,Loved it!,1
4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer a question correctly but Alexa says you got it wrong and answers the same as you. I like being able to turn lights on and off while away from home.",1
5,31-Jul-18,Charcoal Fabric,"I have had a lot of fun with this thing. My 4 yr old learns about dinosaurs, i control the lights and play games like categories. Has nice sound when playing music as well.",1
5,31-Jul-18,Charcoal Fabric,Music,1
5,31-Jul-18,Heather Gray Fabric,"I received the echo as a gift. I needed another Bluetooth or something to play music easily accessible, and found this smart speaker. Can’t wait to see what else it can do.",1
3,31-Jul-18,Sandstone Fabric,"Without having a cellphone, I cannot use many of her features. I have an iPad but do not see that of any use. It IS a great alarm. If u r almost deaf, you can hear her alarm in the bedroom from out in the living room, so that is reason enough to keep her.It is fun to ask random questions to hear her response. She does not seem to be very smartbon politics yet.",1
5,31-Jul-18,Charcoal Fabric,I think this is the 5th one I've purchased. I'm working on getting one in every room of my house. I really like what features they offer specifily playing music on all Echos and controlling the lights throughout my house.,1
5,30-Jul-18,Heather Gray Fabric,looks great,1
5,30-Jul-18,Heather Gray Fabric,"Love it! I’ve listened to songs I haven’t heard since childhood! I get the news, weather, information! It’s great!",1


In [4]:
#display(df) #generar gráfica

In [5]:
#Creamos un RDD a partir de un archivo csv
data_file = "/FileStore/tables/amazon_alexa.csv"
raw_rdd = sc.textFile(data_file).cache()
raw_rdd.take(5) #show the top 5 lines of the file

Out[28]: ['rating,date,variation,verified_reviews,feedback',
 '5,31-Jul-18,Charcoal Fabric ,Love my Echo!,1',
 '5,31-Jul-18,Charcoal Fabric ,Loved it!,1',
 '4,31-Jul-18,Walnut Finish ,"Sometimes while playing a game, you can answer a question correctly but Alexa says you got it wrong and answers the same as you. I like being able to turn lights on and off while away from home.",1',
 '5,31-Jul-18,Charcoal Fabric ,"I have had a lot of fun with this thing. My 4 yr old learns about dinosaurs, i control the lights and play games like categories. Has nice sound when playing music as well.",1']

In [6]:
type(raw_rdd)#se imprime el tipo de clase del argumento pasado como parámetro

Out[29]: pyspark.rdd.RDD

In [7]:
#Vamos a imprimir el 3er registro del RDD
lista = raw_rdd.collect()
print("El tercer elemento de la lista es: %d", lista[3])

El tercer elemento de la lista es: %d 4,31-Jul-18,Walnut Finish ,"Sometimes while playing a game, you can answer a question correctly but Alexa says you got it wrong and answers the same as you. I like being able to turn lights on and off while away from home.",1

In [8]:
#¿Cuantos elementos tiene el RDD?
print("Visualizando el total del elementos:", raw_rdd.count())

Visualizando el total del elementos: 3151

In [9]:
#En RDD, es necesario separar cada una de las entradas, antes de parsear y construir un dataframe
csv_rdd = raw_rdd.map(lambda row: row.split(",")) 
print(csv_rdd.take(3))#print 3 rows

[['rating', 'date', 'variation', 'verified_reviews', 'feedback'], ['5', '31-Jul-18', 'Charcoal Fabric ', 'Love my Echo!', '1'], ['5', '31-Jul-18', 'Charcoal Fabric ', 'Loved it!', '1']]

In [10]:
#Creamos la tabla parsed_rdd.
#Vamos a importar la clase Row de la librería de pyspark.sql


from pyspark.sql import Row #Importamos la clase Row 

amazon_rdd = csv_rdd.map(lambda r: Row(
    rating = r[0],
    date = r[1],
    variation = r[2],
    verified_reviews = r[3],
    feedback = r[4]
    )
)
amazon_rdd.take(5)

Out[33]: [Row(date='date', feedback='feedback', rating='rating', variation='variation', verified_reviews='verified_reviews'),
 Row(date='31-Jul-18', feedback='1', rating='5', variation='Charcoal Fabric ', verified_reviews='Love my Echo!'),
 Row(date='31-Jul-18', feedback='1', rating='5', variation='Charcoal Fabric ', verified_reviews='Loved it!'),
 Row(date='31-Jul-18', feedback=' you can answer a question correctly but Alexa says you got it wrong and answers the same as you. I like being able to turn lights on and off while away from home."', rating='4', variation='Walnut Finish ', verified_reviews='"Sometimes while playing a game'),
 Row(date='31-Jul-18', feedback=' i control the lights and play games like categories. Has nice sound when playing music as well."', rating='5', variation='Charcoal Fabric ', verified_reviews='"I have had a lot of fun with this thing. My 4 yr old learns about dinosaurs')]

In [11]:
#Transformación map: retorna un nuevo RDD
temp_table_name = "amazon_data"
df.createOrReplaceTempView(temp_table_name)

#Construimos un datraframe
df = sqlContext.createDataFrame(amazon_rdd)
display(df.head(10))

date,feedback,rating,variation,verified_reviews
date,feedback,rating,variation,verified_reviews
31-Jul-18,1,5,Charcoal Fabric,Love my Echo!
31-Jul-18,1,5,Charcoal Fabric,Loved it!
31-Jul-18,"you can answer a question correctly but Alexa says you got it wrong and answers the same as you. I like being able to turn lights on and off while away from home.""",4,Walnut Finish,"""Sometimes while playing a game"
31-Jul-18,"i control the lights and play games like categories. Has nice sound when playing music as well.""",5,Charcoal Fabric,"""I have had a lot of fun with this thing. My 4 yr old learns about dinosaurs"
31-Jul-18,1,5,Charcoal Fabric,Music
31-Jul-18,"and found this smart speaker. Can’t wait to see what else it can do.""",5,Heather Gray Fabric,"""I received the echo as a gift. I needed another Bluetooth or something to play music easily accessible"
31-Jul-18,I cannot use many of her features. I have an iPad but do not see that of any use. It IS a great alarm. If u r almost deaf,3,Sandstone Fabric,"""Without having a cellphone"
31-Jul-18,1,5,Charcoal Fabric,I think this is the 5th one I've purchased. I'm working on getting one in every room of my house. I really like what features they offer specifily playing music on all Echos and controlling the lights throughout my house.
30-Jul-18,1,5,Heather Gray Fabric,looks great


In [12]:
reviews_rdd = df.select("verified_reviews").rdd.flatMap(lambda x: x)
#reviews_rdd.collect()

In [13]:
#Remover cabeceras y convertir todos los datos a minúsculas para un fácil procesamiento
header = reviews_rdd.first() #devuelve el primer registro del RDD
data_rmv_col = reviews_rdd.filter(lambda row: row != header) #removemos el encabezado
lowerCase_sentRDD = data_rmv_col.map(lambda x : x.lower()) #convertirmos el texto a minúscula

print(lowerCase_sentRDD.count())
#print(lowerCase_sentRDD.collect())

3150

In [14]:
#Vamos a instalar NLTK en Databricks

In [15]:
%sh 
pip install nltk
pip install --upgrade pip
python -m nltk.downloader all

Requirement already satisfied: nltk in /databricks/python3/lib/python3.7/site-packages (3.4.5)
Requirement already satisfied: six in /databricks/python3/lib/python3.7/site-packages (from nltk) (1.12.0)
Requirement already up-to-date: pip in /databricks/python3/lib/python3.7/site-packages (20.0.2)
/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
 warn(RuntimeWarning(msg))
[nltk_data] Downloading collection 'all'
[nltk_data] | 
[nltk_data] | Downloading package abc to /root/nltk_data...
[nltk_data] | Package abc is already up-to-date!
[nltk_data] | Downloading package alpino to /root/nltk_data...
[nltk_data] | Package alpino is already up-to-date!
[nltk_data] | Downloading package biocreative_ppi to
[nltk_data] | /root/nltk_data...
[nltk_data] | Package biocreative_ppi is already up-to-date!
[nltk_data] | Downloading package brown to /root/nltk_data...
[nltk_data] | Package brown is already up-to-date!
[nltk_data] | Downloading package brown_tei to /root/nltk_data...
[nltk_data] | Package brown_tei is already up-to-date!
[nltk_data] | Downloading package cess_cat to /root/nltk_data...
[nltk_data] | Package cess_cat is already up-to-date!
[nltk_data] | Downloading package cess_esp to /root/nltk_data...
[nltk_data] | Package cess_esp is already up-to-date!
[nltk_data] | Downloading package chat80 to /root/nltk_data...
[nltk_data] | Package chat80 is already up-to-date!
[nltk_data] | Downloading package city_database to
[nltk_data] | /root/nltk_data...
[nltk_data] | Package city_database is already up-to-date!
[nltk_data] | Downloading package cmudict to /root/nltk_data...
[nltk_data] | Package cmudict is already up-to-date!
[nltk_data] | Downloading package comparative_sentences to
[nltk_data] | /root/nltk_data...
[nltk_data] | Package comparative_sentences is already up-to-
[nltk_data] | date!
[nltk_data] | Downloading package comtrans to /root/nltk_data...
[nltk_data] | Package comtrans is already up-to-date!
[nltk_data] | Downloading package conll2000 to /root/nltk_data...
[nltk_data] | Package conll2000 is already up-to-date!
[nltk_data] | Downloading package conll2002 to /root/nltk_data...
[nltk_data] | Package conll2002 is already up-to-date!
[nltk_data] | Downloading package conll2007 to /root/nltk_data...
[nltk_data] | Package conll2007 is already up-to-date!
[nltk_data] | Downloading package crubadan to /root/nltk_data...
[nltk_data] | Package crubadan is already up-to-date!
[nltk_data] | Downloading package dependency_treebank to
[nltk_data] | /root/nltk_data...
[nltk_data] | Package dependency_treebank is already up-to-date!
[nltk_data] | Downloading package dolch to /root/nltk_data...
[nltk_data] | Package dolch is already up-to-date!
[nltk_data] | Downloading package europarl_raw to
[nltk_data] | /root/nltk_data...
[nltk_data] | Package europarl_raw is already up-to-date!
[nltk_data] | Downloading package floresta to /root/nltk_data...
[nltk_data] | Package floresta is already up-to-date!
[nltk_data] | Downloading package framenet_v15 to
[nltk_data] | /root/nltk_data...
[nltk_data] | Package framenet_v15 is already up-to-date!
[nltk_data] | Downloading package framenet_v17 to
[nltk_data] | /root/nltk_data...
[nltk_data] | Package framenet_v17 is already up-to-date!
[nltk_data] | Downloading package gazetteers to /root/nltk_data...
[nltk_data] | Package gazetteers is already up-to-date!
[nltk_data] | Downloading package genesis to /root/nltk_data...
[nltk_data] | Package genesis is already up-to-date!
[nltk_data] | Downloading package gutenberg to /root/nltk_data...
[nltk_data] | Package gutenberg is already up-to-date!
[nltk_data] | Downloading package ieer to /root/nltk_data...
[nltk_data] | Package ieer is already up-to-date!
[nltk_data] | Downloading package inaugural to /root/nltk_data...
[nltk_data] | Package inaugural is already up-to-date!
[nltk_data] | D

In [16]:
#Tokenizamos los textos en sentencias
import nltk
def sent_TokenizeFunct(x):
    return nltk.sent_tokenize(x)
  
sentenceTokenizeRDD = lowerCase_sentRDD.map(sent_TokenizeFunct)
lista = sentenceTokenizeRDD.collect()
print(lista[0:3])

[['love my echo!'], ['loved it!'], ['"sometimes while playing a game']]

In [17]:
#Hacer un split a cada sentencia, para trabajar a nivel palabra 
def word_TokenizeFunct(x):
    splitted = [word for line in x for word in line.split()]
    return splitted
  
wordTokenizeRDD = sentenceTokenizeRDD.map(word_TokenizeFunct)
lista2 = wordTokenizeRDD.collect()
print(lista2[0:3])

[['love', 'my', 'echo!'], ['loved', 'it!'], ['"sometimes', 'while', 'playing', 'a', 'game']]

In [18]:
#Removemos stopwords
def removeStopWordsFunct(x):
    from nltk.corpus import stopwords
    stop_words=set(stopwords.words('english'))
    filteredSentence = [w for w in x if not w in stop_words]
    return filteredSentence
stopwordRDD = wordTokenizeRDD.map(removeStopWordsFunct)
lista3 = stopwordRDD.collect()
print(lista3[0:3])

[['love', 'echo!'], ['loved', 'it!'], ['"sometimes', 'playing', 'game']]

In [19]:
print(stopwordRDD.first())

['love', 'echo!']

In [20]:
#Eliminamos signos de puntuación y espacios en blanco
import string
def removePunctuationsFunct(x):
    list_punct=list(string.punctuation)
    filtered = [''.join(c for c in s if c not in list_punct) for s in x] 
    filtered_space = [s for s in filtered if s] #remove empty space 
    return filtered
rmvPunctRDD = stopwordRDD.map(removePunctuationsFunct)
lista3 = rmvPunctRDD.collect()
print(lista3[0:3])

[['love', 'echo'], ['loved', 'it'], ['sometimes', 'playing', 'game']]

In [21]:
#Análisis de sentimientos
def sentimentWordsFunct(x):
  from nltk.sentiment.vader import SentimentIntensityAnalyzer
  analyzer = SentimentIntensityAnalyzer() 
  senti_list_temp = []
  for i in x:
    y = ''.join(i) 
    vs = analyzer.polarity_scores(y)
    senti_list_temp.append((y, vs))
    senti_list_temp = [w for w in senti_list_temp if w]
  sentiment_list  = []
  for j in senti_list_temp:
    first = j[0]
    second = j[1]
    
    for (k,v) in second.items():
      if k == 'compound':
        if v < 0.0:
          sentiment_list.append((first, "Negative"))
        elif v == 0.0:
          sentiment_list.append((first, "Neutral"))
        else:
          sentiment_list.append((first, "Positive"))
  return sentiment_list

sentimentRDD = rmvPunctRDD.map(sentimentWordsFunct)
sentimentRDD.take(10)

Out[44]: [[('love', 'Positive'), ('echo', 'Neutral')],
 [('loved', 'Positive'), ('it', 'Neutral')],
 [('sometimes', 'Neutral'), ('playing', 'Positive'), ('game', 'Neutral')],
 [('i', 'Neutral'),
 ('lot', 'Neutral'),
 ('fun', 'Positive'),
 ('thing', 'Neutral'),
 ('4', 'Neutral'),
 ('yr', 'Neutral'),
 ('old', 'Neutral'),
 ('learns', 'Neutral'),
 ('dinosaurs', 'Neutral')],
 [('music', 'Neutral')],
 [('i', 'Neutral'),
 ('received', 'Neutral'),
 ('echo', 'Neutral'),
 ('gift', 'Positive'),
 ('needed', 'Neutral'),
 ('another', 'Neutral'),
 ('bluetooth', 'Neutral'),
 ('something', 'Neutral'),
 ('play', 'Positive'),
 ('music', 'Neutral'),
 ('easily', 'Positive'),
 ('accessible', 'Neutral')],
 [('without', 'Neutral'), ('cellphone', 'Neutral')],
 [('think', 'Neutral'),
 ('5th', 'Neutral'),
 ('one', 'Neutral'),
 ('ive', 'Neutral'),
 ('purchased', 'Neutral'),
 ('im', 'Neutral'),
 ('working', 'Neutral'),
 ('getting', 'Neutral'),
 ('one', 'Neutral'),
 ('every', 'Neutral'),
 ('room', 'Neutral'),
 ('house', 'Neutral'),
 ('really', 'Neutral'),
 ('like', 'Positive'),
 ('features', 'Neutral'),
 ('offer', 'Neutral'),
 ('specifily', 'Neutral'),
 ('playing', 'Positive'),
 ('music', 'Neutral'),
 ('echos', 'Neutral'),
 ('controlling', 'Neutral'),
 ('lights', 'Neutral'),
 ('throughout', 'Neutral'),
 ('house', 'Neutral')],
 [('looks', 'Neutral'), ('great', 'Positive')],
 [('love', 'Positive'),
 ('it', 'Neutral'),
 ('i’ve', 'Neutral'),
 ('listened', 'Neutral'),
 ('songs', 'Neutral'),
 ('haven’t', 'Neutral'),
 ('heard', 'Neutral'),
 ('since', 'Neutral'),
 ('childhood', 'Neutral'),
 ('get', 'Neutral'),
 ('news', 'Neutral')]]

In [22]:
#Vamos a extrer la frecuencia de las palabras más usadas
freqDistRDD = rmvPunctRDD.flatMap(lambda x : nltk.FreqDist(x).most_common()).map(lambda x: x).reduceByKey(lambda x,y : x+y).sortBy(lambda x: x[1], ascending = False)

freqDistRDD.take(10)

Out[45]: [('love', 812),
 ('echo', 576),
 ('great', 574),
 ('it', 386),
 ('alexa', 366),
 ('like', 358),
 ('music', 316),
 ('use', 315),
 ('works', 313),
 ('', 293)]

In [23]:
#Vamos a visualizar las palabras más frecuentes
import matplotlib as plt
df_fDist = freqDistRDD.toDF() #convertirmos el RDD hacia un dataframe
df_fDist.createOrReplaceTempView("myTable") 
df2 = spark.sql("SELECT _1 AS PalabrasClave, _2 as Frecuencia from myTable limit 20") #renombramos las columnas
pandas_D = df2.toPandas() #convertimos el dataframe de spark a dataframe de pandas
pandas_D.plot.barh(x='PalabrasClave', y='Frecuencia', rot=1, figsize=(10,8))
display()

In [24]:
display(pandas_D.head(20))

PalabrasClave,Frecuencia
love,812
echo,576
great,574
it,386
alexa,366
like,358
music,316
use,315
works,313
,293


In [25]:
#Guardamos la tabla de manera permanente para su posterior uso
#permanent_table_name = "sentimientos_csv"
#df_fDist.write.format("parquet").saveAsTable(permanent_table_name)